In [11]:
# ---------------------------------------------------------------------------------------
# (C) Vidushi Sharma, 2022
# email: Vidushi@kth.se
# Demonstrates: 
# (1) Extracts and Plot Composite Light curve of Fermi: NaI, BGO, LLE and LAT detectors
# (2) Binning for time-resolved analysis
# (3) Writing the bins to PHAII type file for spectral analysis
# ----------------------------------------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc,rcParams
from astropy.io import fits
from numpy import arange

#%matplotlib inline
#%matplotlib notebook
import glob
from itertools import zip_longest

# Download the data from on-line Fermi catalog using 3ML commands

In [12]:
%%capture
import os

#np.seterr(all="ignore")

from threeML import *
from threeML.io.package_data import get_path_of_data_file

from astropy.time import Time
get_available_plugins()

In [13]:
#Examining the catalog
Grb_name = "GRB160821857"
Trig_name = "bn160821857"

gbm_catalog = FermiGBMBurstCatalog()
gbm_catalog.query_sources(Grb_name)

grb_info = gbm_catalog.get_detector_information()[Grb_name]

gbm_detectors = grb_info["detectors"]
source_interval = grb_info["source"]["fluence"]
background_interval = grb_info["background"]["full"]
best_fit_model = grb_info["best fit model"]["fluence"]
model = gbm_catalog.get_model(best_fit_model, "fluence")[Grb_name]
print(gbm_detectors, source_interval, background_interval, best_fit_model)

model

['n6' 'n7' 'n9' 'b1'] 118.530000-161.539000 -46.670000--9.170000,225.000000-251.670000 band


Model summary:
==============

                  N
Point sources     1
Extended sources  0
Particle sources  0

Free parameters (4):
--------------------

                                          value min_value max_value  \
GRB160821857.spectrum.main.Band.K      0.032686      None      None   
GRB160821857.spectrum.main.Band.alpha -1.054573      -1.5       3.0   
GRB160821857.spectrum.main.Band.xp     940.6278      10.0      None   
GRB160821857.spectrum.main.Band.beta  -2.305336      -5.0      -1.6   

                                                 unit  
GRB160821857.spectrum.main.Band.K      keV-1 s-1 cm-2  
GRB160821857.spectrum.main.Band.alpha                  
GRB160821857.spectrum.main.Band.xp                keV  
GRB160821857.spectrum.main.Band.beta                   

Fixed parameters (3):
(abridged. Use complete=True to see all fixed parameters)


Properties (0):
--------------------

(none)


Linked parameters (0):
----------------------

(none)

Independent variables:
----------------------

(none)

Linked functions (0):
----------------------

(none)

In [14]:
# Creating a new directory and downloading the GBM and LLE data into it.
newpath = r'Spec_Dir' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
# Downloading the GBM data
data_dir=os.path.join('Spec_Dir', Grb_name)
dload = download_GBM_trigger_data(Grb_name, detectors=gbm_detectors, destination_directory=data_dir, compress_tte=True)

# Download LLE data
# Check if grb present in catalog: https://heasarc.gsfc.nasa.gov/db-perl/W3Browse/w3table.pl?tablehead=name%3Dfermille&Action=More+Options
data_dir=os.path.join('Spec_Dir', Grb_name)
lle_data = download_LLE_trigger_data(Grb_name, destination_directory=data_dir)

# Fermi Light Curves  (Data: Either from Fermi website Or, 3ML commands)

In [15]:
# Fermi Trigger Time and Bin size information
Trigger_time = 493504474.039   # xtime converter: 2016-08-21 20:34:30.039

# If data is downloaded by 3ML commands 
if(source_interval[8]=='-'):
    start_g = float(source_interval[0 : 8])
    stop_g = float(source_interval[9 : 18])
else:
    start_g = float(source_interval[0 : 10])
    stop_g = float(source_interval[11 : 19])
    
binwidth = 0.64        # In seconds
t_start = start_g - (25.0*binwidth)
t_stop = stop_g + (25.0*binwidth)

print(binwidth, t_start, t_stop)

0.64 102.53 177.539


In [16]:
#------------------------------------------------------------------------
# Making the Fermi GBM light curve 
#------------------------------------------------------------------------

def na_iodide_lc(event_filename, Trigger_time, binwidth, chan_min, chan_max):
    #Open the fits file 
    pha_list = fits.open(event_filename, memmap=True)

    # load the data in separate variable
    pha_data1 = pha_list[1].data

    #Energy channel number 
    Energy_channel = pha_data1['CHANNEL']

    #load the data in separate variable events
    pha_data2 = pha_list[2].data

    PHA =pha_data2['PHA']
    
    Time = pha_data2['TIME']
    T_n0 = Time - Trigger_time

    #Creating dictionary for Time
    d_n0 = {T_n0[n]: n for n in range(len(T_n0))}

    #Defining the good time interval
    GTI_n0=[]
    PHA_gti_n0 =[]

    #Removing all the values below 100 s before and after the trigger time in the T array and storing the corresponding energy values
    ind=[]
    for key in d_n0:
        if key > -100 and key < 244:
            GTI_n0.append(key)
            ind.append(d_n0[key])
            
    #print GTI_n0
    for i in ind:
         PHA_gti_n0.append(PHA[i])
    
    #Defining the desired lc events
    Time_lc =[]
    ind=[]
    for k,i in zip_longest(PHA_gti_n0,range(len(PHA_gti_n0))):
    #for k,i in map(0,PHA_gti_n0,range(len(PHA_gti_n0))):
        if k > chan_min and k < chan_max:   # for 8 keV - 800 kev - 121 
            ind.append(i)         

    for i in ind:
        Time_lc.append(GTI_n0[i])
    
    #Bins info 
    bins =arange(int(min(Time_lc)),int(max(Time_lc)) + binwidth, binwidth)
 
    #Histogram of events times
    counts_n0, edges=np.histogram(Time_lc,bins)

    Tav_n0 =[]
    for i in range(len(edges)-1):
        Tav_n0.append((edges[i] + edges[i+1])/(2.0))
    edge = edges[1:]
    return[Tav_n0, counts_n0/binwidth, edge]

############## NaI detectors #####################
det = gbm_detectors[0:1]
makeitastring = ''.join(map(str, det))
detector1 = makeitastring
det2 = gbm_detectors[1:2]
makeitastring = ''.join(map(str, det2))
detector2 = makeitastring
#Event file names 
event_filename1 = ''.join(glob.glob('Spec_Dir/'+ str(Grb_name) +'/glg_tte_'+detector1+'_'+Trig_name+'_v0*.fit.gz'))
event_filename2 = ''.join(glob.glob('Spec_Dir/'+ str(Grb_name) +'/glg_tte_'+detector2+'_'+Trig_name+'_v0*.fit.gz'))

if(len(gbm_detectors)==3):
    time_cts_1_8_850 = na_iodide_lc(event_filename1, Trigger_time, binwidth, 3, 122)
    time_cts_2_8_850 = na_iodide_lc(event_filename2, Trigger_time, binwidth, 3, 122)
    Total_cts_nai_8_850 = np.add(time_cts_1_8_850[1], time_cts_2_8_850[1])
    
    time_cts_1_10_50 = na_iodide_lc(event_filename1, Trigger_time, binwidth, 6, 32)
    time_cts_2_10_50 = na_iodide_lc(event_filename2, Trigger_time, binwidth, 6, 32)
    Total_cts_nai_10_50 = np.add(time_cts_1_10_50[1], time_cts_2_10_50[1])
    
    time_cts_1_50_300 = na_iodide_lc(event_filename1, Trigger_time, binwidth, 33, 85)
    time_cts_2_50_300 = na_iodide_lc(event_filename2, Trigger_time, binwidth, 33, 85)
    Total_cts_nai_50_300 = np.add(time_cts_1_50_300[1], time_cts_2_50_300[1])
    
elif(len(gbm_detectors)==4):
    det3 = gbm_detectors[2:3]
    makeitastring = ''.join(map(str, det3))
    detector3 = makeitastring
    event_filename3 = ''.join(glob.glob('Spec_Dir/'+ str(Grb_name) +'/glg_tte_'+detector3+'_'+Trig_name+'_v0*.fit.gz'))
    
    time_cts_1_8_850 = na_iodide_lc(event_filename1, Trigger_time, binwidth, 3, 122)
    time_cts_2_8_850 = na_iodide_lc(event_filename2, Trigger_time, binwidth, 3, 122)
    time_cts_3_8_850 = na_iodide_lc(event_filename3, Trigger_time, binwidth, 3, 122)
    Total_cts_nai_8_850 = np.add(np.add(time_cts_1_8_850[1], time_cts_2_8_850[1]), time_cts_3_8_850[1])
    
    time_cts_1_10_50 = na_iodide_lc(event_filename1, Trigger_time, binwidth, 6, 32)
    time_cts_2_10_50 = na_iodide_lc(event_filename2, Trigger_time, binwidth, 6, 32)
    time_cts_3_10_50 = na_iodide_lc(event_filename3, Trigger_time, binwidth, 6, 32)
    Total_cts_nai_10_50 = np.add(np.add(time_cts_1_10_50[1], time_cts_2_10_50[1]), time_cts_3_10_50[1])
    
    time_cts_1_50_300 = na_iodide_lc(event_filename1, Trigger_time, binwidth, 33, 85)
    time_cts_2_50_300 = na_iodide_lc(event_filename2, Trigger_time, binwidth, 33, 85)
    time_cts_3_50_300 = na_iodide_lc(event_filename3, Trigger_time, binwidth, 33, 85)
    Total_cts_nai_50_300 = np.add(np.add(time_cts_1_50_300[1], time_cts_2_50_300[1]), time_cts_3_50_300[1])

print(event_filename1)
print(event_filename2)
print(event_filename3)    

Spec_Dir/GRB160821857/glg_tte_n6_bn160821857_v00.fit.gz
Spec_Dir/GRB160821857/glg_tte_n7_bn160821857_v00.fit.gz
Spec_Dir/GRB160821857/glg_tte_n9_bn160821857_v00.fit.gz


In [17]:
def bgo_lc(event_filename, Trigger_time, binwidth, chan_min, chan_max):
    #Open the fits file 
    pha_list = fits.open(event_filename, memmap=True)

    #Get info of the fits file
    pha_list.info()

    # load the data in separate variable
    pha_data1 = pha_list[1].data

    #Energy channel number 
    Energy_channel = pha_data1['CHANNEL']

    #load the data in separate variable events
    pha_data2 = pha_list[2].data

    #PHA
    PHA =pha_data2['PHA']

    #Time
    Time = pha_data2['TIME']
    
    #Sort time with respect to the triggertime
    T = Time - Trigger_time

    #Creating dictionary for Time
    d = {T[n]: n for n in range(len(T))}

    #Defining the good time interval
    GTI=[]
    PHA_gti=[]

    #Removing all the values below 100 s before and after the trigger time in the T array and storing the corresponding energy values
    ind=[]
    for key in d:
        if key > -100 and key < 244:
            GTI.append(key)
            ind.append(d[key])

    for i in ind:
        PHA_gti.append(PHA[i])


    #Defining the desired lc events
    Time_lc =[]
    ind=[]
    for k,i in zip_longest(PHA_gti,range(len(PHA_gti))):
        if k > chan_min and k < chan_max:                           
            ind.append(i)

    for i in ind:
        Time_lc.append(GTI[i])

    #Bins info 
    bins =arange(int(min(Time_lc)),int(max(Time_lc)) + binwidth, binwidth)
 
    #Histogram of events times
    counts, edges=np.histogram(Time_lc,bins)

    #Display the histogram plot
    #plt.show()
    Tav=[]
    for i in range(len(edges)-1):
        Tav.append((edges[i] + edges[i+1])/2)
        
    bgo_times=Tav
    bgo_counts=counts
    #Write into the ascii file. 
    #ascii.write([Tav, counts], '131004A_0_1s_B1_300keV_40MeV.lc', names=['Time', 'counts/bin'], formats={'Time':'%4.3f','counts/bin':'%4.3d'})
    return(bgo_times,bgo_counts)

###### BGO: Event file name  ############
det_b = gbm_detectors[-1]
makeitastring = ''.join(map(str, det_b))
detector_b = makeitastring

event_filename_bgo = ''.join(glob.glob('Spec_Dir/'+ str(Grb_name) +'/glg_tte_'+detector_b+'_'+Trig_name+'_v0*.fit.gz'))
bgo_lc=bgo_lc(event_filename_bgo, Trigger_time, binwidth, 4, 119)       # Energy range: 250keV-30MeV

Filename: Spec_Dir/GRB160821857/glg_tte_b1_bn160821857_v00.fit.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      37   ()      
  1  EBOUNDS       1 BinTableHDU     58   128R x 3C   [1I, 1E, 1E]   
  2  EVENTS        1 BinTableHDU     48   992999R x 2C   [1D, 1I]   
  3  GTI           1 BinTableHDU     44   1R x 2C   [1D, 1D]   


In [18]:
def latlle_lc(event_filename, Trigger_time, binwidth):
    #Open the fits file 
    pha_list = fits.open(event_filename, memmap=True)

    #Get info of the fits file
    pha_list.info()
    #print(pha_list[1].columns)

    # load the data in separate variable
    pha_data1 = pha_list[1].data

    #LLE, TIME and Energy 
    Time = pha_data1['TIME']
    Energy = pha_data1['ENERGY']

    T = Time - Trigger_time

    #Creating dictionary for Time
    d = {T[n]: n for n in range(len(T))}

    #Defining the good time interval
    GTI=[]
    PHA_gti=[]

    #removing all the values below 100 s before and after the trigger time in the T array and storing the corresponding energy values
    ind=[]
    for key in d:
        if key > -100 and key < 244:
            GTI.append(key)
            ind.append(d[key])
    
    for i in ind:
        PHA_gti.append(Energy[i])
   
    Time_lc =[]
    ind=[]
    for k,i in zip_longest(PHA_gti,range(len(PHA_gti))):
        if 30.0 <= k <= 150.0:   # LLE energy range 30 MeV to 3000 MeV 
            ind.append(i)

    for i in ind:
        Time_lc.append(GTI[i])

    #Bins info 
    bins =arange(int(min(Time_lc)),int(max(Time_lc)) + binwidth, binwidth)
 
    #Histogram of events times
    counts_l, edges=np.histogram(Time_lc,bins)

    Tav_l=[]
    for i in range(len(edges)-1):
        Tav_l.append((edges[i] + edges[i+1])/2)
        
    lle_time=Tav_l
    lle_counts=counts_l
    return(lle_time, lle_counts)

# LAT-LLE Event files  ############
event_filename_lle = ''.join(glob.glob('Spec_Dir/'+ str(Grb_name) +'/gll_lle_'+Trig_name+'_v0*.fit'))
lle_lc = latlle_lc(event_filename_lle, Trigger_time, binwidth)

Filename: Spec_Dir/GRB160821857/gll_lle_bn160821857_v00.fit
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      41   ()      
  1  EVENTS        1 BinTableHDU    120   26348R x 14C   [E, E, E, E, E, E, E, E, E, D, J, J, J, E]   
  2  GTI           1 BinTableHDU     55   1R x 2C   [D, D]   


# Download Fermi-LAT data 

In [19]:
# GRB Info
ra =  171.248    # 308.5970833    
dec =  42.343    # 6.9197222

# Download data from tstart to tstop
tstart = Trigger_time 
tstop = Trigger_time + 1000.0*binwidth

# Note that this will understand if you already download these files, and will not do it twice unless you change your selection or the outdir
evfile, scfile = download_LAT_data(ra, dec, 12.0, tstart, tstop, time_type='MET', destination_directory='Spec_Dir/data_dir_lat')

# Configuration for Fermipy
config = FermipyLike.get_basic_config(evfile=evfile, scfile=scfile, ra=ra, dec=dec)
config['selection']['emax'] = 3000.0 # In MeV 
config['selection']['emin'] = 100.0 # MeV = 0.1 GeV
config['gtlike'] = {'edisp': True}

#Let's create an instance of the plugin Note that here no processing is made, because fermipy still doesn't know about the model you want to use
fermi_lat = FermipyLike("LAT", config)
#config.display()

# LAT Max energy photon Information
#Open the fits file 
pha_list_src = fits.open(evfile, memmap=True)
#print(pha_list_src[1].columns)

# load the data in separate variable
pha_data_src = pha_list_src[1].data

Energy_src = pha_data_src['ENERGY']
print(max(Energy_src)/1000.)      ##GeV units

ImportError: dlopen(/Users/dove/anaconda3/envs/threeML/lib/fermitools/_pyLikelihood.so, 2): Library not loaded: @rpath/libCLHEP-2.4.4.1.dylib
  Referenced from: /Users/dove/anaconda3/envs/threeML/lib/fermitools/_pyLikelihood.so
  Reason: image not found

In [ ]:
def lat_lc(event_filename,Trigger_time,binwidth):
    #Open the fits file 
    pha_list = fits.open(event_filename, memmap=True)

    #Get info of the fits file
    pha_list.info()

    # load the data in separate variable
    pha_data1 = pha_list[1].data

    #LLE, TIME and Energy 
    Time = pha_data1['TIME']
    Energy = pha_data1['ENERGY']
    #print(min(Energy), max(Energy))

    T = Time - Trigger_time

    #Creating dictionary for Time
    d = {T[n]: n for n in range(len(T))}

    #Defining the good time interval
    GTI=[]
    PHA_gti=[]

    #removing all the values below 100 s before and after the trigger time in the T array and storing the corresponding energy values
    ind=[]
    for key in d:
        if key > -100 and key < 244:
            GTI.append(key)
            ind.append(d[key])
    
    for i in ind:
        PHA_gti.append(Energy[i])

    #Defining the desired lc events
    Time_lc =[]
    ind=[]
    for k,i in zip_longest(PHA_gti,range(len(PHA_gti))):
        if 100.0 <= k:   # LAT energy >100 MeV
            ind.append(i)    

    energy=[]
    for i in ind:
        Time_lc.append(GTI[i])
        energy.append(PHA_gti[i]/1000.)
    
    #Bins info 
    bins =arange(int(min(Time_lc)),int(max(Time_lc)) + binwidth, binwidth)

    #Histogram of events times
    counts_l, edges=np.histogram(Time_lc,bins)

    Tav_l=[]
    for i in range(len(edges)-1):
        Tav_l.append((edges[i] + edges[i+1])/2)
    
    lat_time=Tav_l
    lat_counts=counts_l
    lat_energy=energy
    lat_time_lc= Time_lc
    return(lat_time,lat_counts,lat_energy,lat_time_lc)

## LAT event file
event_filename_LAT = ''.join(glob.glob('Spec_Dir/data_dir_lat/*_FT1.fits'))
# It has energy info too!
lat_lc = lat_lc(event_filename_LAT, Trigger_time, binwidth)

# Plotting Fermi light curve

In [ ]:
# Composite Light curve: GBM + LAT-LLE + LAT 
fig_all_lc, (ax1a, ax1a_b, ax2a, ax3a) = plt.subplots(4, sharex=True, sharey=False, figsize=(9,8))  
plt.rc('text', usetex=False)
plt.rc('font', family='times')
hfont = {'fontname':'Times New Roman'}

#  First subplot: NaI detectors 
ax1a.step(time_cts_1_8_850[2], Total_cts_nai_8_850, lw=1.0, color='k', alpha=0.7, label=r'GBM NaI: 8 - 850 keV')
ax1a.errorbar(time_cts_1_8_850[0], Total_cts_nai_8_850, yerr=np.sqrt(Total_cts_nai_8_850), 
              color ='black', fmt='o', ms='2.0', elinewidth=0.5, alpha=0.3)
ax1a.legend(shadow=True, numpoints=1, loc='upper left', fontsize=15)
ax1a.set_ylabel('Counts/bin', fontsize=16, **hfont)
ax1a.fill_between(time_cts_1_8_850[2], 0, Total_cts_nai_8_850, facecolor='k', alpha=0.1)
ax1a.set_xlim([-10, 280])
ax1a.locator_params(axis='y',tight=True, nbins=3)
ax1a.locator_params(axis='x',tight=True, nbins=8)
ax1a.tick_params(axis='both',which='both', width=1,direction='in')
ax1a.tick_params(which='major', length=8)
ax1a.tick_params(which='minor', length=4)
ax1a.tick_params(labelsize=15)
ax1a.tick_params(axis='x', labelsize=16)
ax1a.tick_params(axis='y', labelsize=16)
ax1a.axvline(0.0, color='k', linestyle='--',lw='1.25')
#ax1a.axvline(start_g, color='grey', linestyle=':',lw='1.5')
#ax1a.axvline(stop_g, color='grey', linestyle=':',lw='1.5')
#plt.title(GRBname[grb_no], fontsize=18)

#Second subplot: BGO detector
ax1a_b.step(bgo_lc[0], bgo_lc[1], linewidth=1.0,color='darkblue', alpha=0.7, label=r'GBM BGO: 250 keV - 30 MeV')
ax1a_b.errorbar(bgo_lc[0], bgo_lc[1], yerr=np.sqrt(bgo_lc[1]), 
              color ='darkblue', fmt='o', ms='2.0', elinewidth=0.5, alpha=0.3)
ax1a_b.legend(shadow=True, numpoints=1, loc='upper left', fontsize=15)
ax1a_b.set_ylabel('Counts/bin', fontsize=16, **hfont)
ax1a_b.fill_between(bgo_lc[0], 0, bgo_lc[1], facecolor='darkblue', alpha=0.15)
ax1a_b.set_xlim([-10, 250])
ax1a_b.locator_params(axis='y',tight=True, nbins=3)
ax1a_b.locator_params(axis='x',tight=True, nbins=8)
ax1a_b.tick_params(axis='both',which='both', width=1,direction='in')
ax1a_b.tick_params(which='major', length=8)
ax1a_b.tick_params(which='minor', length=4)
ax1a_b.tick_params(labelsize=15)
ax1a_b.tick_params(axis='x', labelsize=16)
ax1a_b.tick_params(axis='y', labelsize=16)
ax1a_b.axvline(0.0, color='k', linestyle='--',lw='1.25')

#Third subplot: LLE detector
ax2a.step(lle_lc[0], lle_lc[1], linewidth=1.0,color='darkgreen', alpha=0.75, label=r'LAT-LLE: 30 - 150 MeV')
ax2a.errorbar(lle_lc[0], lle_lc[1], yerr=np.sqrt(lle_lc[1]), 
              color ='darkgreen', fmt='o', ms='2.0', elinewidth=0.5, alpha=0.3)
ax2a.legend(shadow=True, numpoints=1, loc='upper left', fontsize=15)
ax2a.set_ylabel('Counts/bin', fontsize=16, **hfont)
ax2a.fill_between(lle_lc[0], 0, lle_lc[1], facecolor='darkgreen', alpha=0.2)
ax2a.set_xlim([-10, 250])
ax2a.locator_params(axis='y',tight=True, nbins=3)
ax2a.locator_params(axis='x',tight=True, nbins=8)
ax2a.tick_params(axis='both',which='both', width=1,direction='in')
ax2a.tick_params(which='major', length=8)
ax2a.tick_params(which='minor', length=4)
ax2a.tick_params(labelsize=15)
ax2a.tick_params(axis='x', labelsize=16)
ax2a.tick_params(axis='y', labelsize=16)
ax2a.axvline(0.0, color='k', linestyle='--',lw='1.25')

#Third subplot: LAT detector
ax3a.plot(lat_lc[0], lat_lc[1],linewidth=1.0, color='firebrick', alpha=0.75, label=r'LAT: 300 MeV - 5 GeV')
ax3a.errorbar(lat_lc[0], lat_lc[1], yerr=np.sqrt(lat_lc[1]), 
              color ='firebrick', fmt='o', ms='3.0', elinewidth=0.5, alpha=0.45)
ax3a.legend(shadow=True, numpoints=1, loc='upper left', fontsize=15)
ax3a.set_ylabel('Counts/bin', fontsize=16, **hfont)
ax3a.set_xlim([-10, 250])
ax3a.fill_between(lat_lc[0], 0, lat_lc[1], facecolor='firebrick', alpha=0.25)
ax3a.locator_params(axis='y',tight=True, nbins=3)
ax3a.locator_params(axis='x',tight=True, nbins=8)
ax3a.tick_params(axis='both',which='both', width=1,direction='in')
ax3a.tick_params(which='major', length=8)
ax3a.tick_params(which='minor', length=4)
ax3a.tick_params(labelsize=15)
ax3a.tick_params(axis='x', labelsize=16)
ax3a.tick_params(axis='y', labelsize=16)
ax3a.axvline(0.0, color='k', linestyle='--',lw='1.25')
ax3a.set_xlabel('Time Since GBM Trigger [s]',fontsize=16,**hfont)

fig_all_lc.subplots_adjust(hspace=0.025)
plt.setp([a.get_xticklabels() for a in fig_all_lc.axes[:-1]], visible=True)
fig_all_lc.savefig('Spec_Dir/'+Grb_name+'_Fermi_Lightcurve.pdf', bbox_inches='tight')
plt.show()

In [ ]:
def HR_error(Total_cts_low, Total_cts_high):
    HR_gbm = []
    HR_gbm_err = []            
    for i in range(len(Total_cts_low)):
        HR = Total_cts_high[i]/float(Total_cts_low[i])
        HR_gbm.append(HR)
        error = np.sqrt(HR**2.0 * ((np.sqrt(Total_cts_high[i])/float(Total_cts_high[i]))**2.0 + (np.sqrt(Total_cts_low[i])/float(Total_cts_low[i])**2.0))  )
        HR_gbm_err.append(error)
    return [HR_gbm, HR_gbm_err]

gbm_counts = np.add(Total_cts_nai_8_850, bgo_lc[1])
    
HR_gbm_lle = HR_error(gbm_counts, lle_lc[1])[0]
HR_gbm_lle_err = HR_error(gbm_counts, lle_lc[1])[1]

fig_hr, (ax1a) = plt.subplots(1, sharex=True, sharey=False,figsize=(8,7))  
plt.rc('text', usetex=False)
plt.rc('font', family='times')
hfont = {'fontname':'Times New Roman'}

#fig.suptitle(GRB_name, fontsize=18)
ax1a.step(time_cts_1_8_850[2], HR_gbm_lle, lw=1.25, color='k', alpha=0.75)
ax1a.errorbar(time_cts_1_8_850[0], HR_gbm_lle, yerr=HR_gbm_lle_err, fmt='ko', ms=7.0, elinewidth=1.0, alpha=0.6, label=r'(30 - 150 MeV)/(8 keV - 30 MeV)')
ax1a.legend(shadow=True, numpoints=1, loc='upper right', fontsize=16)
ax1a.fill_between(time_cts_1_8_850[0], 0, HR_gbm_lle, facecolor='k', alpha=0.05)
ax1a.set_xlim([110, 180])
ax1a.locator_params(axis='y',tight=True, nbins=4)
ax1a.locator_params(axis='x',tight=True, nbins=10)
ax1a.tick_params(axis='both',which='both', width=1, direction='in')
ax1a.tick_params(which='major', length=8)
ax1a.tick_params(which='minor', length=4)
ax1a.tick_params(labelsize=15)
ax1a.tick_params(axis='x', labelsize=16)
ax1a.tick_params(axis='y', labelsize=16)
ax1a.set_ylabel('Hardness Ratio (LLE/GBM)',fontsize=18, **hfont)
ax1a.set_xlabel('Time Since GBM Trigger [s]',fontsize=18,**hfont)

fig_hr.subplots_adjust(hspace=0.025)
plt.setp([a.get_xticklabels() for a in fig_hr.axes[:-1]], visible=True)
fig_hr.savefig('Spec_Dir/'+Grb_name+'_HR_LLE_GBM.pdf')

plt.show()

# Time-series and Spectral Plugins

In [ ]:
#source_interval= str("180.000000-250.000000")

# Building plugins from GBM
fluence_plugins = []
time_series = {}
for det in gbm_detectors:

    ts_cspec = TimeSeriesBuilder.from_gbm_cspec_or_ctime(
        det, cspec_or_ctime_file=dload[det]["cspec"], rsp_file=dload[det]["rsp"]
    )

    ts_cspec.set_background_interval(*background_interval.split(","))
    ts_cspec.save_background(f"{det}_bkg.h5", overwrite=True)

    ts_tte = TimeSeriesBuilder.from_gbm_tte(
        det,
        tte_file=dload[det]["tte"],
        rsp_file=dload[det]["rsp"],
        restore_background=f"{det}_bkg.h5",
    )

    time_series[det] = ts_tte

    ts_tte.set_active_time_interval(source_interval)

    ts_tte.view_lightcurve(t_start-10., t_stop+10.)

    fluence_plugin = ts_tte.to_spectrumlike()
    
    fluence_plugin.display()  # For significance in each detector

    if det.startswith("b"):

        fluence_plugin.set_active_measurements("300-30000")

    else:

        fluence_plugin.set_active_measurements("8-30","40-850")

    fluence_plugin.rebin_on_background(1.0)

    fluence_plugins.append(fluence_plugin)


In [ ]:
# Building plugins from LAT-LLE
lle_file = event_filename_lle
ft2_file = ''.join(glob.glob('Spec_Dir/'+ str(Grb_name) +'/gll_pt_'+Trig_name+'_v00.fit'))
lle_rsp = ''.join(glob.glob('Spec_Dir/'+ str(Grb_name) +'/gll_cspec_'+Trig_name+'_v00.rsp'))

lat_lle = TimeSeriesBuilder.from_lat_lle('lat_lle',
                                        lle_file=lle_file,
                                        ft2_file=ft2_file,
                                        rsp_file=lle_rsp)

#selecting source intervals
lat_lle.set_active_time_interval(str(source_interval))

#Fitting a polynomial background
lat_lle.set_background_interval("-50--10", "200-245", unbinned=False)   #-87.500000--22.500000,817.500000-890.000000

#Saving the background fit
lat_lle.save_background('background_store',overwrite=True)

fig = lat_lle.view_lightcurve(start=100, stop=190);
_ = plt.axvline(x=122, color='g')
_ = plt.axvline(x=136, color='g')
_ = plt.axvline(x=146, color='g')

lle_plugin = lat_lle.to_spectrumlike()
lle_plugin.display()

# Time Resolved Analysis

In [ ]:
#Selecting a temporal binning
nai = time_series["n6"]

nai.create_time_bins(110.0, 170.0, method="bayesblocks", use_background=True, p0=0.01)
#nai.create_time_bins(180.0, 250.0, method="significance", sigma=25.0)

fig_bb = nai.view_lightcurve(use_binner=True)

In [ ]:
bad_bins = []
for i, w in enumerate(nai.bins.widths):

    if w < 5e-2:
        bad_bins.append(i)

edges = [nai.bins.starts[0]]

for i, b in enumerate(nai.bins):

    if i not in bad_bins:
        edges.append(b.stop)

starts = edges[:-1]
stops = edges[1:]
#print(starts[0], stops[0])
nai.bins.display()

fig_bb = nai.view_lightcurve(110.0, 170.0, use_binner=True)
fig_bb.savefig('Spec_Dir/'+Grb_name+'_Bayes_Block_LC.pdf')

#s_int= str(starts[2])+'-'+str(stops[2])
#nai.set_active_time_interval(s_int)
#check_plugin = nai.to_spectrumlike(from_bins=True)

# Significance of brightest NaI and LLE detectors

In [ ]:
#check_plugin[0].display()
#import pandas as pd
#df = pd.DataFrame(check_plugin[0])
#sig = df.values[1]

from threeML.utils.statistics.stats_tools import Significance
#===== Store in a file =======
file_b = open('Spec_Dir/'+Grb_name+'_Each_Interval_Significance.txt', 'a')
file_b.write('Interval    Start        Stop        Midpoint        Significance' + '\n')

for j in range(len(starts)):
    mid_p = (starts[j]+stops[j])/2.
    total_counts = nai._time_series.counts_over_interval(starts[j], stops[j])
    bkg_counts = nai._time_series.get_total_poly_count(starts[j], stops[j])
    bkg_error = nai._time_series.get_total_poly_error(starts[j], stops[j])
    sig_calc = Significance(total_counts, bkg_counts)
    sig_val = sig_calc.li_and_ma_equivalent_for_gaussian_background(bkg_error)[0]
    file_b.write('%d' %j + '\t %2.4f' %starts[j] + '\t %2.4f' %stops[j] + '\t %2.4f' %mid_p + '\t %2.2f' %sig_val +'\n')
file_b.close()

In [ ]:
#===== Store in a file =======
file_l = open('Spec_Dir/'+Grb_name+'_Each_Interval_Significance_LLE.txt', 'a')
file_l.write('Interval    Start        Stop        Midpoint        Significance' + '\n')

for j in range(len(starts)):
    mid_p = (starts[j]+stops[j])/2.
    total_counts = lat_lle._time_series.counts_over_interval(starts[j], stops[j])
    bkg_counts = lat_lle._time_series.get_total_poly_count(starts[j], stops[j])
    bkg_error = lat_lle._time_series.get_total_poly_error(starts[j], stops[j])
    sig_calc = Significance(total_counts, bkg_counts)
    sig_val = sig_calc.li_and_ma_equivalent_for_gaussian_background(bkg_error)[0]
    file_l.write('%d' %j + '\t %2.4f' %starts[j] + '\t %2.4f' %stops[j] + '\t %2.4f' %mid_p + '\t %2.2f' %sig_val +'\n')
file_l.close()

# Writing as PHAII file for analysis

In [ ]:
#Selecting a temporal binning
nai.create_time_bins(start = starts, stop = stops, method="custom")
nai.write_pha_from_binner(file_name='Spec_Dir/Fermi_GBM_nai_bright', overwrite=True,
                              force_rsp_write = True)  # if you need to write the RSP to a file. We try to choose the best option for you.

nai_2 = time_series['n7']
nai_2.create_time_bins(start = starts, stop = stops, method="custom")
nai_2.write_pha_from_binner(file_name='Spec_Dir/Fermi_GBM_nai_2', overwrite=True,
                              force_rsp_write = True)  

nai_3 = time_series['n9']
nai_3.create_time_bins(start = starts, stop = stops, method="custom")
nai_3.write_pha_from_binner(file_name='Spec_Dir/Fermi_GBM_nai_3', overwrite=True,
                              force_rsp_write = True)  

nai_b = time_series['b1']
nai_b.create_time_bins(start = starts, stop = stops, method="custom")
nai_b.write_pha_from_binner(file_name='Spec_Dir/Fermi_GBM_bgo_1', overwrite=True,
                              force_rsp_write = True)  

#Selecting a temporal binning
lat_lle.create_time_bins(start = starts, stop = stops, method='custom')
lat_lle.write_pha_from_binner(file_name='Spec_Dir/Fermi_LAT_lle', overwrite=True,
                              force_rsp_write = True)  # if you need to write the RSP to a file. We try to choose the best option for you.

In [ ]:
time_resolved_plugins = {}

for k, v in time_series.items():
    v.read_bins(nai)
    time_resolved_plugins[k] = v.to_spectrumlike(from_bins=True)
    
lle_plugins = lat_lle.to_spectrumlike(from_bins=True)

# Time-series and Spectral Plugins For EAC Test

In [10]:
#Selecting a temporal binning
nai = time_series["n6"]

#nai.create_time_bins(110.0, 170.0, method="bayesblocks", use_background=True, p0=0.01)
nai.create_time_bins(180.0, 250.0, method="significance", sigma=30.0)

fig_bb = nai.view_lightcurve(use_binner=True)

NameError: name 'time_series' is not defined